#### Make gif from .tif images

In [ ]:
import os
import re
import rasterio
import numpy as np
import imageio
from PIL import Image

# Path to directory containing the .tif files
base_path = "../code/results/output/e6a08fb8/tree_shade/27"
output_gif_path = "../code/results/output/visuals/e6a08fb8/shade_over_day.gif"

# Regular expression pattern to extract timestamp from filenames
pattern = re.compile(r".*_(\d{8})_(\d{4})_LST\.tif")  # Example: file_20230305_1200_LST.tif

# Collect valid .tif files
frame_files = []
for filename in os.listdir(base_path):
    if filename.endswith(".tif") and not filename.endswith(".tif.ovr"):  # Exclude overview files
        match = pattern.match(filename)
        if match:
            frame_files.append(os.path.join(base_path, filename))

# Sort files by date and time extracted from filename
frame_files.sort()

# List to store processed images
frames = []

# Read and process each `.tif` file
for tif_file in frame_files:
    with rasterio.open(tif_file) as src:
        # Read the first band (if multiband, modify as needed)
        img_array = src.read(1)

        # Normalize to 0-255 if necessary
        img_array = (255 * (img_array - np.min(img_array)) / (np.max(img_array) - np.min(img_array))).astype(np.uint8)

        # Convert to PIL image
        img = Image.fromarray(img_array)
        frames.append(img)

# Save as GIF
frames[0].save(output_gif_path, save_all=True, append_images=frames[1:], duration=100, loop=0)

print(f"GIF saved at: {output_gif_path}")


#### Make .gif from plots

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import imageio
import os

# Define output folder for frames
output_folder = "grid_animation_frames"
os.makedirs(output_folder, exist_ok=True)

# Plot the base layer: points dataset
fig, ax = plt.subplots(figsize=(10, 8))
points_dataset.plot(ax=ax, color="red", markersize=0.4, label="Points")
plt.title("Grid Cells Processing")
plt.xlabel("East (m)")
plt.ylabel("North (m)")
plt.grid(True)

# Save initial frame (just points dataset)
frame_files = []
frame_path = os.path.join(output_folder, f"frame_000.png")
plt.savefig(frame_path, dpi=150)
frame_files.append(frame_path)

# Process and overlay tiles one by one
valid_tiles = valid_grid_gdf[valid_grid_gdf["point_count"] >= 5000]

for i, (_, row) in enumerate(valid_tiles.iterrows()):
    row_gdf = gpd.GeoDataFrame([row], crs=valid_tiles.crs)

    # Overlay the current tile
    row_gdf.plot(ax=ax, edgecolor="blue", facecolor="none", linewidth=1.5)

    # Annotate the cell index
    centroid = row.geometry.centroid
    ax.text(centroid.x, centroid.y, str(i), fontsize=8, color="black",
            ha="center", va="center", bbox=dict(facecolor="white", alpha=0.5, edgecolor="none"))

    # Save frame
    frame_path = os.path.join(output_folder, f"frame_{i+1:03d}.png")
    plt.savefig(frame_path, dpi=150)
    frame_files.append(frame_path)

# Create GIF
gif_path = "../code/results/output/visuals/d48092b5/tile_processing.gif"
imageio.mimsave(gif_path, [imageio.imread(frame) for frame in frame_files], duration=0.5)

print(f"GIF saved at: {gif_path}")